# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from matplotlib.patches import ConnectionPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','LateLesion.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

---

plotting group bar plots

In [ ]:
def plot_event_rec(root, ax, Profiles, badAnimals=None, TaskParamToPlot='% good trials',
                   x_pos=0, nPre=3, nPost=3, nRec=3, n_bootstraps=10000,**kwarg):

    if badAnimals is None:
        badAnimals=[]
        
    if x_pos==0:
        diff=.35
        x_c=ax.get_xlim()[1]
        x_pos=(x_c-diff,x_c,x_c+diff)
    
    #getting the data
    _,SessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)

    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])

    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values()))
    
    xPre= np.nanmedian(data[:,nSessionPre-nPre:nSessionPre],axis=1)
    xPost=np.nanmedian(data[:,nSessionPre:nPost+nSessionPre],axis=1)
    xRec=[]
    for i in range(data.shape[0]):
        d=data[i,:]
        try:
            ind=np.where(~np.isnan(d) == True)[0][-1]
        except IndexError:
            xRec.appen(np.nan)
            continue
        xRec.append(np.nanmedian(d[ind-nRec:]))
    xRec=np.array(xRec)
    
    _xPre =bootstrap(xPre, n_bootstraps=n_bootstraps, user_statistic=lambda x:np.nanmedian(x,axis=1))
    _xPost=bootstrap(xPost, n_bootstraps=n_bootstraps, user_statistic=lambda x:np.nanmedian(x,axis=1))
    _xRec =bootstrap(xRec, n_bootstraps=n_bootstraps, user_statistic=lambda x:np.nanmedian(x,axis=1))
    
    x=np.vstack([_xPre,_xPost,_xRec]).T
    
    y=np.nanpercentile(x,50,axis=0)
    yerr=np.nanpercentile(x,(5,95),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='o', ms=2, elinewidth=1, **kwarg)
    
    

#     ax.set_xlim([-1,1])
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['-1','+1','rec'])
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([0,100])
    ax.set_ylabel(TaskParamToPlot)

    return x

def add_legend_for_lesion_types(ax):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='tomato', marker='o',
                                       markerfacecolor='tomato', markersize=3, label='DLS')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:blue', marker='o',
                                       markerfacecolor='xkcd:blue', markersize=3, label='DMS')
    m_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='magenta', marker='o',
                                       markerfacecolor='magenta', markersize=3, label='DS')
    k_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='k', marker='o',
                                       markerfacecolor='k', markersize=3, label='Control')

    errorbar = matplotlib.lines.Line2D([0,0], [0,1], color='k', label='95% CI')


    leg=ax.legend(handles=[k_marker,r_marker,b_marker,m_marker,errorbar],loc=8,
                  facecolor=None,edgecolor=None, fontsize='xx-small',frameon=False)
    return leg

In [ ]:
if "__file__" not in dir():
    #the inputs
    fig=plt.figure(figsize=(8,1.5))
    gs= fig.add_gridspec(nrows=1, ncols=4, left=0.02, bottom=0.02, right=0.98, top=.98, wspace=.1)
    

    
    profile1pre={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control']
             }
    
    profile1post={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control-AfterBreak']
             }
    animalList1Ctrl='Rat250'
    
    sessionIndex1pre =(0,-1)
    sessionIndex1post=(0,-1)
    
    plot_pre_post_traj(root, gs, animalList1Ctrl, profile1pre, profile1post, sessionIndex1pre, sessionIndex1post)
    gs.figure.add_artist(ConnectionPatch(xyA=(.5,0), xyB=(.5,1), coordsA='figure fraction', coordsB='figure fraction',
                                     ls='--',lw=2))

    
    plt.show()
    plt.close('all')

---

plotting the predefined image

In [ ]:
def plot_animal_image(ax, animal):
    PATHS=('/NAS02/Rat250/Histology/DLS_25.jpg',
           '/NAS02/Rat217/Histology/DS_35.jpg',
           '/NAS02/Rat304/Histology/DS_35.jpg')
    
    try:
        filePath=[path for path in PATHS if animal in path][0]
    except IndexError:
        logging.error(f'Bad Animal name ({animal}), path not defined!')
        return
    
    f=PIL.Image.open(filePath)
    f.thumbnail((500,500),PIL.Image.ANTIALIAS)
    ax.imshow(f)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GRID 1 PARAMS
    
    profile1pre={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control']
             }
    
    profile1post={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control-AfterBreak']
             }
    
    animalList1Ctrl='Rat250'
    
    sessionIndex1pre =(0,-1)
    sessionIndex1post=(0,5)
    

    
    #===============================================
    
    # GRID 2 PARAMS
    
    profile2pre={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control-AfterBreak']
             }
    
    profile2post={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Late-Lesion_DLS']
             }
    
    animalList2Ctrl='Rat250'
    sessionIndex2post=(0,2)
    
    
    #================================================
    
    # GRID 3 PARAMS
    
    profile3pre={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control-AfterBreak']
             }
    
    profile3post={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Late-Lesion_DMS']
             }
    
    animalList3Ctrl='Rat217'
    sessionIndex3post=(0,4)

    
    #================================================
    
    # GRID 4 PARAMS
    
    profile4pre={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control-Sharp']
             }
    
    profile4post={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Late-Lesion_DS-Sharp']
             }
    
    animalList4Ctrl='Rat304'
    sessionIndex4post=(0,6)


Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(4,4)
    fig=plt.figure(figsize=figsize,dpi=600)
    
    
    ##########################################
    # 1: Control
    gs1= fig.add_gridspec(nrows=1, ncols=4, left=0.02, bottom=0.63, right=0.99, top=.78)
    
    axes1=plot_pre_post_traj(root, gs1, animalList1Ctrl, profile1pre, profile1post,
                             sessionIndex1pre, sessionIndex1post,showText=True)
    
    axes1[0].set_title('start',fontsize='x-small')
    axes1[1].set_title('#-1',fontsize='x-small')
    axes1[2].set_title('#+1',fontsize='x-small')
    axes1[3].set_title(f'recovered(#+{sessionIndex1post[-1]+1})',fontsize='x-small')
    
    
    #adding the vertical line + 'Break' text
    gs1.figure.add_artist(ConnectionPatch(xyA=(.5,0), xyB=(.5,gs1.top), coordsA='figure fraction', coordsB='figure fraction',
                                     ls='--',lw=1.1))

    gs1.figure.add_artist(mpl.text.Text(x=.5, y=gs1.top, text='Break', c='k',ha='center',va='bottom',fontsize='x-small'))
    
    
    
    ##########################################
    # 2: DLS
    gs2= fig.add_gridspec(nrows=1, ncols=4, left=0.02, bottom=0.4, right=0.99, top=.55)
    
    axes2=plot_pre_post_traj(root, gs2, animalList2Ctrl, profile2pre, profile2post, sessionIndex1pre, sessionIndex2post)
    
    axes2[0].clear()
    plot_animal_image(axes2[0], animalList2Ctrl)
    axes2[0].set_ylabel('DLS')
    axes2[3].set_title(f'#+{sessionIndex2post[-1]+1}',fontsize='x-small')
    
    
    
    ##########################################
    # 3: DMS
    gs3= fig.add_gridspec(nrows=1, ncols=4, left=0.02, bottom=0.2, right=0.99, top=.35)
    
    axes3=plot_pre_post_traj(root, gs3, animalList3Ctrl, profile3pre, profile3post, sessionIndex1pre, sessionIndex3post)
    
    axes3[0].clear()
    plot_animal_image(axes3[0], animalList3Ctrl)
    axes3[0].set_ylabel('DMS')
    axes3[3].set_title(f'#+{sessionIndex3post[-1]+1}',fontsize='x-small')
    

    ##########################################
    # 4: DS
    gs4= fig.add_gridspec(nrows=1, ncols=4, left=0.02, bottom=0.0, right=0.99, top=.15)
    
    axes4=plot_pre_post_traj(root, gs4, animalList4Ctrl, profile4pre, profile4post, sessionIndex1pre, sessionIndex4post)
    
    axes4[0].clear()
    plot_animal_image(axes4[0], animalList4Ctrl)
    axes4[0].set_ylabel('DS')
    axes4[3].set_title(f'#+{sessionIndex4post[-1]+1}',fontsize='x-small')
    

    
    fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(axes4[0],ax1,ax2,ax5,ax7,ax8,ax6)
#     OFFX=np.array([.07]*len(AXES))
#     OFFY=np.array([.01]*len(AXES))
#     OFFX[5]=0.03
#     OFFX[[0,1,2,4,6]]=0.05
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LateLesion.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()